In [8]:
!pip install -q sentence-transformers transformers

import re, numpy as np
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Load models
embedder = SentenceTransformer("all-MiniLM-L6-v2")
PARAPHRASER_MODEL = "Vamsi/T5_Paraphrase_Paws"
tokenizer = AutoTokenizer.from_pretrained(PARAPHRASER_MODEL)
model_seq2seq = AutoModelForSeq2SeqLM.from_pretrained(PARAPHRASER_MODEL)
paraphraser = pipeline("text2text-generation", model=model_seq2seq, tokenizer=tokenizer, device=-1)

# Reference lists
strong_ctas = ["Buy Now", "Shop Now", "Learn More", "Sign Up", "Get Started", "Join Now", "Subscribe"]
engaging_phrases = ["limited time offer", "exclusive deal", "save big today", "🔥 hot sale", "best choice for you"]
blocklist = ["scam", "$$$", "clickbait", "fake", "100% free money"]

# --- Helpers ---
def normalize_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'([!?.])\1+', r'\1', text)
    return text

def smart_truncate(text, max_len):
    """Truncate at word boundary, not mid-word."""
    if len(text) <= max_len:
        return text
    return text[:max_len].rsplit(" ", 1)[0]

def truncate_text(text, channel, kind):
    if channel.lower() == "google":
        if kind == "title":
            return smart_truncate(text, 30)
        elif kind == "description":
            return smart_truncate(text, 90)
    return text

def remove_blocklist(text):
    for bad in blocklist:
        text = re.sub(re.escape(bad), "", text, flags=re.IGNORECASE)
    return text

# --- NLP scoring ---
def semantic_score(text, reference_list):
    if not text.strip(): return 0
    embeddings = embedder.encode([text] + reference_list, convert_to_tensor=True)
    sim = util.cos_sim(embeddings[0], embeddings[1:]).cpu().numpy()
    return float(np.max(sim))

def rate_title_nlp(title):
    length = len(title.split())
    length_score = 0.7 if 5 <= length <= 10 else 0.3
    engagement = semantic_score(title, engaging_phrases)
    return round((length_score + engagement) * 5, 2)

def rate_desc_nlp(desc, channel):
    length = len(desc.split())
    length_score = 0.7 if 15 <= length <= 30 else 0.3
    cta_score = semantic_score(desc, strong_ctas)
    engage_score = semantic_score(desc, engaging_phrases)
    channel_bonus = 0.7 if channel.lower() in ["instagram","tiktok"] and "🔥" in desc else 0.5
    return round((length_score + cta_score + engage_score + channel_bonus) * 2.5, 2)

def rate_cta_nlp(cta):
    return round(semantic_score(cta, strong_ctas) * 10, 2)

# --- Channel style enforcement ---
def channel_style_rules(text, channel):
    if channel.lower() == "linkedin":
        return re.sub(r"[🔥😍👍]", "", text)  # remove emojis
    if channel.lower() in ["instagram","tiktok"] and not re.search(r"[🔥😍👍]", text):
        text += " 🔥"  # add emoji if missing
    return text

# --- Templates ---
def template_improvements(text, channel, kind="title"):
    text = normalize_text(remove_blocklist(text))
    if kind == "title":
        if channel.lower() == "google":
            return [
                f"{text} | 50% Off",
                f"{text} Deals",
                f"{text} Official Site"
            ]
        elif channel.lower() in ["instagram","tiktok"]:
            return [f"🔥 {text} — Limited Time!", f"{text} 😍 Don’t Miss Out"]
        elif channel.lower() == "linkedin":
            return [f"{text} — Professional Solutions", f"Discover {text} for Business"]
        else:
            return [f"{text} — Shop Now", f"{text} — Learn More"]
    else:
        if channel.lower() == "google":
            return [
                f"{text}. Order Now. Fast Shipping.",
                f"{text}. Save More Today.",
                f"{text}. Limited Time Deals."
            ]
        elif channel.lower() in ["instagram","tiktok"]:
            return [f"{text}. Hurry 🔥", f"{text}. Shop Now 😍"]
        elif channel.lower() == "linkedin":
            return [f"{text}. Drive Business Growth.", f"{text}. Contact Us for More."]
        else:
            return [f"{text}. Buy Now & Save.", f"{text}. Explore More."]

# --- Paraphraser with guardrails ---
def improve_text_safe(text, channel="generic", kind="title", num_return=2):
    text = normalize_text(remove_blocklist(text))
    if not text.strip():
        return []
    if len(text.split()) <= 2:
        return template_improvements(text, channel, kind)
    try:
        outputs = paraphraser(
            f"paraphrase: {text} </s>",
            max_new_tokens=32,
            do_sample=True,
            top_p=0.9,
            temperature=0.8,
            repetition_penalty=1.3,
            num_return_sequences=num_return
        )
        results = []
        for out in outputs:
            gen = normalize_text(out["generated_text"].replace("paraphrase:", "").strip())
            gen = channel_style_rules(gen, channel)
            gen = truncate_text(gen, channel, kind)
            if 3 <= len(gen.split()) <= 20:
                results.append(gen)
        if not results:
            return template_improvements(text, channel, kind)
        return list(dict.fromkeys(results))[:num_return]
    except:
        return template_improvements(text, channel, kind)

# --- Main evaluation ---
def evaluate_ad_nlp(channel, title, description, cta):
    title_score = rate_title_nlp(title)
    desc_score = rate_desc_nlp(description, channel)
    cta_score = rate_cta_nlp(cta)
    channel_fit = np.mean([title_score, desc_score])
    final_score = round((title_score + desc_score + cta_score + channel_fit) / 4, 2)

    feedback, improvements = [], {}
    if title_score < 7:
        feedback.append("Headline is weak — try making it sharper.")
        improvements["title"] = improve_text_safe(title, channel, "title")
    if desc_score < 7:
        feedback.append("Description is weak — add urgency or clearer CTA.")
        improvements["description"] = improve_text_safe(description, channel, "description")
    if cta_score < 7:
        feedback.append("CTA is weak — make it more action-driven.")
        improvements["cta"] = improve_text_safe(cta, channel, "cta")

    return {
        "channel": channel,
        "scores": {
            "title": title_score,
            "description": desc_score,
            "cta": cta_score,
            "channel_fit": round(channel_fit,2),
            "final": final_score
        },
        "feedback": feedback,
        "improvements": improvements
    }

# --- Interactive input ---
print("=== Ad Text Evaluation Tool ===")
channel = input("Enter marketing channel (Instagram, Google, LinkedIn, TikTok, Email, etc.): ") or "Instagram"
title = input("Enter Ad Title / Headline: ") or "Big Sale Today"
description = input("Enter Ad Description: ") or "Get your favorite products at half price today only!"
cta = input("Enter Call To Action (CTA): ") or "Shop Now"

# Run evaluation
result = evaluate_ad_nlp(channel, title, description, cta)

# Print results
print("\n===== Results =====")
print("Channel:", result["channel"])
print("Scores:", result["scores"])
print("Final Score:", result["scores"]["final"])
print("\nFeedback:")
for fb in result["feedback"]:
    print("-", fb)
if result["improvements"]:
    print("\nSuggested Improvements:")
    for field, suggestions in result["improvements"].items():
        print(f"{field.capitalize()} suggestions:")
        for s in suggestions:
            print("  •", s)


Device set to use cpu


=== Ad Text Evaluation Tool ===
Enter marketing channel (Instagram, Google, LinkedIn, TikTok, Email, etc.): Google
Enter Ad Title / Headline: Affordable Prices on Summer Wear
Enter Ad Description: Get trendy outfits at half price this week only. Limited stock available 🔥 Grab yours today!
Enter Call To Action (CTA): Shop Now

===== Results =====
Channel: Google
Scores: {'title': 5.02, 'description': 5.09, 'cta': 10.0, 'channel_fit': np.float64(5.06), 'final': np.float64(6.29)}
Final Score: 6.29

Feedback:
- Headline is weak — try making it sharper.
- Description is weak — add urgency or clearer CTA.

Suggested Improvements:
Title suggestions:
  • Affordable prices on summer
  • Affordable prices on Summer
Description suggestions:
  • Get trendy outfits at half price this week only Limited stock available Grab yours today!
  • This week only get trendy outfits at half price - limited stock available Grab yours
